# SHSAT Test Results Preliminary EDA / Cleaning Notebook
[Return to project overview](final_project_overview.ipynb)

### Andrew Larimer, Deepak Nagaraj, Daniel Olmstead, Michael Winton (W207-4-Summer 2018 Final Project)

The [NYC SHSAT Test Results 2017](https://www.kaggle.com/willkoehrsen/nyc-shsat-test-results-2017/home) dataset includes...

In [ ]:
# import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# set default options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

%matplotlib inline

In [ ]:
# load dataset from CSV

In [ ]:
# do stuff to clean it up

In [ ]:
# show the EDA plots / stats that seem most interesting

In [ ]:
# save the cleaned dataset to CSV
# shsat_cleaned_df.to_csv('shsat_cleaned.csv')